## import needed packages

In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
import pprint
import pandas as pd
from scipy.stats import beta
from pprint import pprint
from easydict import EasyDict as edict

## functions 

In [2]:
def sortf(f):
    num = f.name.split("_")[-2].split("p")[-1]
    num = int(num)
    return  num


def load_pkl(f):
    with open(f, "rb") as fi:
        data = pickle.load(fi)
    return data


def is_valid(dat, num=1000):
    numsps1 = len(dat["UIPm_sps"]["sps"])
    numsps2 = len(dat["UIPKL_sps"]["sps"])
    return (numsps1 > num) and (numsps2 > num)


def is_true(p0, method=None, dat=None, bs=None):
    assert (dat is None) + (bs is None) == 1
    if dat is not None:
        res = dat[method]
        low, up = res["eq"]
    else:
        low, up = bs
    return (p0 > low) and (p0 < up)


def rejrates(p0, data):
    ress = {"jef":[], "full":[], "jpp":[], "UIPKL":[], "UIPm":[]}
    for dat in data:
        for method in ress.keys():
            ress[method].append(is_true(p0, dat, method))
    for res in ress.items():
        keyv, reslist = res
        ress[keyv] = 1 - np.mean(reslist)
    return ress

def rejrate(p0, data, q):
    if len(data[0]) > 2:
        reslist = [is_true(p0, bs=[np.quantile(dat, q=q), np.quantile(dat, q=1-q)]) for dat in data] 
    else:
        reslist = [is_true(p0, bs=[beta.ppf(q=q, a=dat[0], b=dat[1]), beta.ppf(q=1-q, a=dat[0], b=dat[1])]) for dat in data] 
    return 1 - np.mean(reslist)


def getRatio(p0, data=None, para=None):
    assert (data is None) + (para is None) == 1
    if para is not None:
        a, b = para
        rv = beta(a=a, b=b)
        res = np.min([rv.cdf(p0), rv.sf(p0)])
    if data is not None:
        p1 = np.mean(data<=p0)
        p2 = np.mean(data>p0)
        res = np.min([p1, p2])
    return res



def getQuantile(p0, data=None, paras=None, alp=0.05):
    assert (data is None) + (paras is None) == 1
    if paras is not None:
        res = [getRatio(p0, para=para) for para in paras]
    if data is not None:
        res = [getRatio(p0, data=dat) for dat in data]
    #n = len(res)
    return np.quantile(res, q=alp)


## n = 40 

In [3]:
n = 40
root = Path(f"./betaMCMC1000n{n}nsdiff/")
files = root.glob("*.pkl")
files = list(files)
# sort the files
files = sorted(files, key=sortf, reverse=False)
filesnpp = [f for f in files if "NPP" in f.name]
files = [f for f in files if "NPP" not in f.name]

# test p = p0
idxs = [0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]
p0 = 0.50

f = files[idxs.index(p0)]
fnpp = filesnpp[idxs.index(p0)]

### get the calibrated quantile

In [4]:
data = load_pkl(f)
datanpp = load_pkl(fnpp)
dicdata = edict()
cvqs = edict()

fulldata = [dat["full_popu"] for dat in data]
JEFdata = [dat["jef_popu"] for dat in data]
JPPdata = [dat["jpp_sps"]["sps"]  for dat in data]
NPPdata = [dat["npp_sps"]["theta"]  for dat in datanpp]
UIPDdata = [dat["UIPD_sps"]["sps"]  for dat in data]
UIPJSdata = [dat["UIPKL_sps"]["sps"]  for dat in data]

dicdata.FULL = fulldata
dicdata.JEF = JEFdata 
dicdata.JPP = JPPdata 
dicdata.NPP = NPPdata 
dicdata.UIPD = UIPDdata 
dicdata.UIPJS = UIPJSdata 

In [5]:
for key, v in dicdata.items():
    if key in ["FULL", "JEF"]:
        cvqs[key] = getQuantile(p0, paras=v)
    else:
        cvqs[key] = getQuantile(p0, data=v)
        
cvqs.JEF = cvqs.JEF * 0.9
cvqs.NPP = cvqs.NPP * 0.98
cvqs.UIPD = cvqs.UIPD * 0.96
cvqs.UIPJS = cvqs.UIPJS * 0.98

In [10]:
Sizes = edict()
for key, qv in cvqs.items():
    Sizes[key] = rejrate(p0, dicdata[key], q=qv)
pprint(Sizes)

{'FULL': 0.04600000000000004,
 'JEF': 0.04500000000000004,
 'JPP': 0.050000000000000044,
 'NPP': 0.050000000000000044,
 'UIPD': 0.050000000000000044,
 'UIPJS': 0.050000000000000044}


In [11]:
cvqs

{'FULL': 1.3879890025103592e-06,
 'JEF': 0.02538278667550111,
 'JPP': 0.023756,
 'NPP': 0.002744,
 'UIPD': 0.00384,
 'UIPJS': 0.00392}

### calibrate the q manually

In [12]:
key = "FULL"
rejrate(p0, dicdata[key], q=cvqs[key]*1.01)

0.05900000000000005

### Obtain powers

In [14]:
powers = []
powers95 = []
for pklfile, nppfile in zip(files, filesnpp):
    p = sortf(pklfile)/100
    if p == p0:
        data = load_pkl(pklfile)
        datanpp = load_pkl(nppfile)
        JEFdata = [dat["jef_popu"] for dat in data]
        fulldata = [dat["full_popu"] for dat in data]
        UIPDdata = [dat["UIPD_sps"]["sps"]  for dat in data]
        UIPKLdata = [dat["UIPKL_sps"]["sps"]  for dat in data]
        JPPdata = [dat["jpp_sps"]["sps"]  for dat in data]
        NPPdata = [dat["npp_sps"]["theta"]  for dat in datanpp]
    
        Sizes95 = {
            "FULL": rejrate(p0, fulldata, q=0.025),
            "JEF": rejrate(p0, JEFdata, q=0.025),
            "JPP": rejrate(p0, JPPdata, q=0.025),
            "NPP": rejrate(p0, NPPdata, q=0.025),
            "UIPD": rejrate(p0, UIPDdata, q=0.025),
            "UIPJS": rejrate(p0, UIPKLdata, q=0.025),
            "p0": p
            }
    else:
        data = load_pkl(pklfile)
        datanpp = load_pkl(nppfile)
        JEFdata = [dat["jef_popu"] for dat in data]
        fulldata = [dat["full_popu"] for dat in data]
        UIPDdata = [dat["UIPD_sps"]["sps"]  for dat in data]
        UIPKLdata = [dat["UIPKL_sps"]["sps"]  for dat in data]
        JPPdata = [dat["jpp_sps"]["sps"]  for dat in data]
        NPPdata = [dat["npp_sps"]["theta"]  for dat in datanpp]
    
        res = {
            "FULL": rejrate(p0, fulldata, q=cvqs.FULL),
            "JEF": rejrate(p0, JEFdata, q=cvqs.JEF),
            "JPP": rejrate(p0, JPPdata, q=cvqs.JPP),
            "NPP": rejrate(p0, NPPdata, q=cvqs.NPP),
            "UIPD": rejrate(p0, UIPDdata, q=cvqs.UIPD),
            "UIPJS": rejrate(p0, UIPKLdata, q=cvqs.UIPJS),
            "p0": p
            }
        res95 = {
            "FULL": rejrate(p0, fulldata, q=0.025),
            "JEF": rejrate(p0, JEFdata, q=0.025),
            "JPP": rejrate(p0, JPPdata, q=0.025),
            "NPP": rejrate(p0, NPPdata, q=0.025),
            "UIPD": rejrate(p0, UIPDdata, q=0.025),
            "UIPJS": rejrate(p0, UIPKLdata, q=0.025),
            "p0": p
            }
        powers.append(res) 
        powers95.append(res95) 

In [15]:
powers = pd.DataFrame(powers)
powers95 = pd.DataFrame(powers95)
print(powers95)
print(powers)

print(f"Powers")
print(powers.drop(columns=["p0"]).mean(axis=0))
print(f"Powers 95")
print(powers95.drop(columns=["p0"]).mean(axis=0))

    FULL    JEF    JPP    NPP   UIPD  UIPJS    p0
0  0.999  0.693  0.791  0.965  0.958  0.946  0.30
1  0.992  0.423  0.512  0.879  0.845  0.831  0.35
2  0.963  0.208  0.297  0.723  0.658  0.623  0.40
3  0.953  0.075  0.117  0.464  0.398  0.368  0.45
    FULL    JEF    JPP    NPP   UIPD  UIPJS    p0
0  0.378  0.693  0.762  0.755  0.795  0.835  0.30
1  0.288  0.423  0.489  0.517  0.573  0.611  0.35
2  0.175  0.208  0.273  0.321  0.331  0.356  0.40
3  0.104  0.075  0.099  0.132  0.145  0.156  0.45
Powers
FULL     0.23625
JEF      0.34975
JPP      0.40575
NPP      0.43125
UIPD     0.46100
UIPJS    0.48950
dtype: float64
Powers 95
FULL     0.97675
JEF      0.34975
JPP      0.42925
NPP      0.75775
UIPD     0.71475
UIPJS    0.69200
dtype: float64


In [16]:
print("Size")
for key, v in Sizes.items():
    print(f"{key:<10}: {v:.3f}")
print("Size95")
for key, v in Sizes95.items():
    print(f"{key:<10}: {v:.3f}")

Size
FULL      : 0.046
JEF       : 0.045
JPP       : 0.050
NPP       : 0.050
UIPD      : 0.050
UIPJS     : 0.050
Size95
FULL      : 0.929
JEF       : 0.045
JPP       : 0.058
NPP       : 0.256
UIPD      : 0.193
UIPJS     : 0.163
p0        : 0.500


## n = 80

In [17]:
n = 80
root = Path(f"./betaMCMC1000n{n}nsdiff/")
files = root.glob("*.pkl")
files = list(files)
# sort the files
files = sorted(files, key=sortf, reverse=False)
filesnpp = [f for f in files if "NPP" in f.name]
files = [f for f in files if "NPP" not in f.name]

# test p = p0
idxs = [0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]
p0 = 0.50

f = files[idxs.index(p0)]
fnpp = filesnpp[idxs.index(p0)]

data = load_pkl(f)
datanpp = load_pkl(fnpp)
dicdata = edict()
cvqs = edict()

fulldata = [dat["full_popu"] for dat in data]
JEFdata = [dat["jef_popu"] for dat in data]
JPPdata = [dat["jpp_sps"]["sps"]  for dat in data]
NPPdata = [dat["npp_sps"]["theta"]  for dat in datanpp]
UIPDdata = [dat["UIPD_sps"]["sps"]  for dat in data]
UIPJSdata = [dat["UIPKL_sps"]["sps"]  for dat in data]

dicdata.FULL = fulldata
dicdata.JEF = JEFdata 
dicdata.JPP = JPPdata 
dicdata.NPP = NPPdata 
dicdata.UIPD = UIPDdata 
dicdata.UIPJS = UIPJSdata 

In [18]:
for key, v in dicdata.items():
    if key in ["FULL", "JEF"]:
        cvqs[key] = getQuantile(p0, paras=v)
    else:
        cvqs[key] = getQuantile(p0, data=v)
        
cvqs.NPP = cvqs.NPP * 0.95
cvqs.UIPD = cvqs.UIPD * 0.95
cvqs.UIPJS = cvqs.UIPJS * 0.95

In [19]:
Sizes = edict()
for key, qv in cvqs.items():
    Sizes[key] = rejrate(p0, dicdata[key], q=qv)
pprint(Sizes)

{'FULL': 0.05500000000000005,
 'JEF': 0.062000000000000055,
 'JPP': 0.050000000000000044,
 'NPP': 0.050000000000000044,
 'UIPD': 0.050000000000000044,
 'UIPJS': 0.050000000000000044}


### calibrate the q manually

In [20]:
key = "UIPJS"
rejrate(p0, dicdata[key], q=cvqs[key]*0.95)

0.04500000000000004

### Obtain powers

In [21]:
powers = []
powers95 = []
for pklfile, nppfile in zip(files, filesnpp):
    p = sortf(pklfile)/100
    if p == p0:
        data = load_pkl(pklfile)
        datanpp = load_pkl(nppfile)
        JEFdata = [dat["jef_popu"] for dat in data]
        fulldata = [dat["full_popu"] for dat in data]
        UIPDdata = [dat["UIPD_sps"]["sps"]  for dat in data]
        UIPKLdata = [dat["UIPKL_sps"]["sps"]  for dat in data]
        JPPdata = [dat["jpp_sps"]["sps"]  for dat in data]
        NPPdata = [dat["npp_sps"]["theta"]  for dat in datanpp]
        Sizes95 = {
            "FULL": rejrate(p0, fulldata, q=0.025),
            "JEF": rejrate(p0, JEFdata, q=0.025),
            "JPP": rejrate(p0, JPPdata, q=0.025),
            "NPP": rejrate(p0, NPPdata, q=0.025),
            "UIPD": rejrate(p0, UIPDdata, q=0.025),
            "UIPJS": rejrate(p0, UIPKLdata, q=0.025),
            "p0": p
            }
    else:
        data = load_pkl(pklfile)
        datanpp = load_pkl(nppfile)
        JEFdata = [dat["jef_popu"] for dat in data]
        fulldata = [dat["full_popu"] for dat in data]
        UIPDdata = [dat["UIPD_sps"]["sps"]  for dat in data]
        UIPKLdata = [dat["UIPKL_sps"]["sps"]  for dat in data]
        JPPdata = [dat["jpp_sps"]["sps"]  for dat in data]
        NPPdata = [dat["npp_sps"]["theta"]  for dat in datanpp]
    
        res = {
            "FULL": rejrate(p0, fulldata, q=cvqs.FULL),
            "JEF": rejrate(p0, JEFdata, q=cvqs.JEF),
            "JPP": rejrate(p0, JPPdata, q=cvqs.JPP),
            "NPP": rejrate(p0, NPPdata, q=cvqs.NPP),
            "UIPD": rejrate(p0, UIPDdata, q=cvqs.UIPD),
            "UIPJS": rejrate(p0, UIPKLdata, q=cvqs.UIPJS),
            "p0": p
            }
        res95 = {
            "FULL": rejrate(p0, fulldata, q=0.025),
            "JEF": rejrate(p0, JEFdata, q=0.025),
            "JPP": rejrate(p0, JPPdata, q=0.025),
            "NPP": rejrate(p0, NPPdata, q=0.025),
            "UIPD": rejrate(p0, UIPDdata, q=0.025),
            "UIPJS": rejrate(p0, UIPKLdata, q=0.025),
            "p0": p
            }
        powers.append(res) 
        powers95.append(res95) 

In [22]:
powers = pd.DataFrame(powers)
powers95 = pd.DataFrame(powers95)
print(powers95)
print(powers)

print(f"Powers")
print(powers.drop(columns=["p0"]).mean(axis=0))
print(f"Powers 95")
print(powers95.drop(columns=["p0"]).mean(axis=0))

    FULL    JEF    JPP    NPP   UIPD  UIPJS    p0
0  0.999  0.962  0.962  0.997  0.996  0.991  0.30
1  0.998  0.817  0.817  0.963  0.954  0.938  0.35
2  0.987  0.456  0.456  0.808  0.771  0.710  0.40
3  0.943  0.141  0.141  0.505  0.446  0.387  0.45
    FULL    JEF    JPP    NPP   UIPD  UIPJS    p0
0  0.686  0.962  0.962  0.967  0.971  0.968  0.30
1  0.500  0.817  0.817  0.834  0.862  0.841  0.35
2  0.273  0.456  0.456  0.509  0.536  0.504  0.40
3  0.132  0.141  0.141  0.196  0.215  0.191  0.45
Powers
FULL     0.39775
JEF      0.59400
JPP      0.59400
NPP      0.62650
UIPD     0.64600
UIPJS    0.62600
dtype: float64
Powers 95
FULL     0.98175
JEF      0.59400
JPP      0.59400
NPP      0.81825
UIPD     0.79175
UIPJS    0.75650
dtype: float64


In [23]:
print("Size")
for key, v in Sizes.items():
    print(f"{key:<10}: {v:.3f}")
print("Size95")
for key, v in Sizes95.items():
    print(f"{key:<10}: {v:.3f}")

Size
FULL      : 0.055
JEF       : 0.062
JPP       : 0.050
NPP       : 0.050
UIPD      : 0.050
UIPJS     : 0.050
Size95
FULL      : 0.848
JEF       : 0.062
JPP       : 0.051
NPP       : 0.232
UIPD      : 0.163
UIPJS     : 0.138
p0        : 0.500


## n=120

In [24]:
n = 120
root = Path(f"./betaMCMC1000n{n}nsdiff/")
files = root.glob("*.pkl")
files = list(files)
# sort the files
files = sorted(files, key=sortf, reverse=False)
filesnpp = [f for f in files if "NPP" in f.name]
files = [f for f in files if "NPP" not in f.name]

# test p = p0
idxs = [0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]
p0 = 0.50

f = files[idxs.index(p0)]
fnpp = filesnpp[idxs.index(p0)]

data = load_pkl(f)
datanpp = load_pkl(fnpp)
dicdata = edict()
cvqs = edict()

fulldata = [dat["full_popu"] for dat in data]
JEFdata = [dat["jef_popu"] for dat in data]
JPPdata = [dat["jpp_sps"]["sps"]  for dat in data]
NPPdata = [dat["npp_sps"]["theta"]  for dat in datanpp]
UIPDdata = [dat["UIPD_sps"]["sps"]  for dat in data]
UIPJSdata = [dat["UIPKL_sps"]["sps"]  for dat in data]

dicdata.FULL = fulldata
dicdata.JEF = JEFdata 
dicdata.JPP = JPPdata 
dicdata.NPP = NPPdata 
dicdata.UIPD = UIPDdata 
dicdata.UIPJS = UIPJSdata 

In [25]:
for key, v in dicdata.items():
    if key in ["FULL", "JEF"]:
        cvqs[key] = getQuantile(p0, paras=v)
    else:
        cvqs[key] = getQuantile(p0, data=v)
        
cvqs.NPP = cvqs.NPP * 0.98
cvqs.UIPD = cvqs.UIPD * 0.99
cvqs.UIPJS = cvqs.UIPJS * 0.98
cvqs.FULL = cvqs.FULL * 1.01

In [26]:
Sizes = edict()
for key, qv in cvqs.items():
    Sizes[key] = rejrate(p0, dicdata[key], q=qv)
pprint(Sizes)

{'FULL': 0.05600000000000005,
 'JEF': 0.05900000000000005,
 'JPP': 0.050000000000000044,
 'NPP': 0.050000000000000044,
 'UIPD': 0.050000000000000044,
 'UIPJS': 0.050000000000000044}


### calibrate the q manually

In [27]:
key = "FULL"
rejrate(p0, dicdata[key], q=cvqs[key]*1.01)

0.05600000000000005

### Obtain powers

In [28]:
powers = []
powers95 = []
for pklfile, nppfile in zip(files, filesnpp):
    p = sortf(pklfile)/100
    if p == p0:
        data = load_pkl(pklfile)
        datanpp = load_pkl(nppfile)
        JEFdata = [dat["jef_popu"] for dat in data]
        fulldata = [dat["full_popu"] for dat in data]
        UIPDdata = [dat["UIPD_sps"]["sps"]  for dat in data]
        UIPKLdata = [dat["UIPKL_sps"]["sps"]  for dat in data]
        JPPdata = [dat["jpp_sps"]["sps"]  for dat in data]
        NPPdata = [dat["npp_sps"]["theta"]  for dat in datanpp]
        Sizes95 = {
            "FULL": rejrate(p0, fulldata, q=0.025),
            "JEF": rejrate(p0, JEFdata, q=0.025),
            "JPP": rejrate(p0, JPPdata, q=0.025),
            "NPP": rejrate(p0, NPPdata, q=0.025),
            "UIPD": rejrate(p0, UIPDdata, q=0.025),
            "UIPJS": rejrate(p0, UIPKLdata, q=0.025),
            "p0": p
            }
    else:
        data = load_pkl(pklfile)
        datanpp = load_pkl(nppfile)
        JEFdata = [dat["jef_popu"] for dat in data]
        fulldata = [dat["full_popu"] for dat in data]
        UIPDdata = [dat["UIPD_sps"]["sps"]  for dat in data]
        UIPKLdata = [dat["UIPKL_sps"]["sps"]  for dat in data]
        JPPdata = [dat["jpp_sps"]["sps"]  for dat in data]
        NPPdata = [dat["npp_sps"]["theta"]  for dat in datanpp]
    
        res = {
            "FULL": rejrate(p0, fulldata, q=cvqs.FULL),
            "JEF": rejrate(p0, JEFdata, q=cvqs.JEF),
            "JPP": rejrate(p0, JPPdata, q=cvqs.JPP),
            "NPP": rejrate(p0, NPPdata, q=cvqs.NPP),
            "UIPD": rejrate(p0, UIPDdata, q=cvqs.UIPD),
            "UIPJS": rejrate(p0, UIPKLdata, q=cvqs.UIPJS),
            "p0": p
            }
        res95 = {
            "FULL": rejrate(p0, fulldata, q=0.025),
            "JEF": rejrate(p0, JEFdata, q=0.025),
            "JPP": rejrate(p0, JPPdata, q=0.025),
            "NPP": rejrate(p0, NPPdata, q=0.025),
            "UIPD": rejrate(p0, UIPDdata, q=0.025),
            "UIPJS": rejrate(p0, UIPKLdata, q=0.025),
            "p0": p
            }
        powers.append(res) 
        powers95.append(res95) 

In [29]:
powers = pd.DataFrame(powers)
powers95 = pd.DataFrame(powers95)
print(powers95)
print(powers)

print(f"Powers")
print(powers.drop(columns=["p0"]).mean(axis=0))
print(f"Powers 95")
print(powers95.drop(columns=["p0"]).mean(axis=0))


    FULL    JEF    JPP    NPP   UIPD  UIPJS    p0
0  1.000  0.995  0.995  1.000  1.000  1.000  0.30
1  0.998  0.928  0.928  0.988  0.985  0.977  0.35
2  0.987  0.562  0.562  0.846  0.807  0.774  0.40
3  0.942  0.184  0.183  0.512  0.451  0.406  0.45
    FULL    JEF    JPP    NPP   UIPD  UIPJS    p0
0  0.917  0.995  0.997  0.999  0.999  0.999  0.30
1  0.760  0.928  0.935  0.947  0.953  0.944  0.35
2  0.425  0.562  0.583  0.644  0.656  0.631  0.40
3  0.176  0.184  0.198  0.249  0.250  0.245  0.45
Powers
FULL     0.56950
JEF      0.66725
JPP      0.67825
NPP      0.70975
UIPD     0.71450
UIPJS    0.70475
dtype: float64
Powers 95
FULL     0.98175
JEF      0.66725
JPP      0.66700
NPP      0.83650
UIPD     0.81075
UIPJS    0.78925
dtype: float64


In [30]:
print("Size")
for key, v in Sizes.items():
    print(f"{key:<10}: {v:.3f}")
print("Size95")
for key, v in Sizes95.items():
    print(f"{key:<10}: {v:.3f}")

Size
FULL      : 0.056
JEF       : 0.059
JPP       : 0.050
NPP       : 0.050
UIPD      : 0.050
UIPJS     : 0.050
Size95
FULL      : 0.771
JEF       : 0.059
JPP       : 0.048
NPP       : 0.166
UIPD      : 0.126
UIPJS     : 0.115
p0        : 0.500
